In [10]:
import pandas as pd
import os

#### EPIC

In [10]:
# GC
print('********* GC *********')
gc = pd.read_csv('data/matrix.tsv', sep='\t', index_col=0)
gc = gc.T
gc = gc[gc.index != 'Sample name']
gc.sort_index(inplace=True)
print(f'Dataset shape: {gc.shape}')
print(f'Number of features: {gc.shape[1]}')
print(f'Number of missing values: {gc.isnull().sum().sum()}')
gc_labels = pd.read_csv('data/batches.tsv', sep='\t', index_col=0)
gc_labels = gc_labels[gc_labels.index != 'Sample name']
print(gc_labels['Factor1'].value_counts())
print('*****************************')  
print()

# LC-MS (+)
print('********* LC-MS (+) *********')
df_np = pd.read_csv('data/EPIC_norm_pos.txt', sep='\t')
df_np.set_index('Samples', inplace=True)
df_np.sort_index(inplace=True)
print(f'Dataset shape: {df_np.shape}')
print(f'Number of features: {df_np.shape[1]-1}')
print(f'Number of missing values: {df_np.isnull().sum().sum()}')
print(df_np['group'].value_counts())
print('*****************************')
print()

# CE-MS
print('********* CE-MS *********')
df_rp = pd.read_csv('data/EPIC_rev_pos.txt', sep='\t')
df_rp.set_index('Samples', inplace=True)
df_rp.sort_index(inplace=True)
print(f'Dataset shape: {df_rp.shape}')
print(f'Number of features: {df_rp.shape[1]-1}')
print(f'Number of missing values: {df_rp.isnull().sum().sum()}')
print(df_rp['group'].value_counts())
print('*****************************')
print()

# LC-MS (-)
print('********* LC-MS (-) *********')
df_rn = pd.read_csv('data/EPIC_rev_neg.txt', sep='\t')
df_rn.set_index('Samples', inplace=True)
df_rn.sort_index(inplace=True)
print(f'Dataset shape: {df_rn.shape}')
print(f'Number of features: {df_rn.shape[1]-1}')
print(f'Number of missing values: {df_rn.isnull().sum().sum()}')
print(df_rn['group'].value_counts())
print('*****************************')
print()

********* GC *********
Dataset shape: (78, 60)
Number of features: 60
Number of missing values: 0
CASE       39
CONTROL    39
Name: Factor1, dtype: int64
*****************************

********* LC-MS (+) *********
Dataset shape: (78, 510)
Number of features: 509
Number of missing values: 2
Factor1:CASE       39
Factor1:CONTROL    39
Name: group, dtype: int64
*****************************

********* CE-MS *********
Dataset shape: (78, 330)
Number of features: 329
Number of missing values: 0
Factor1:CASE       39
Factor1:CONTROL    39
Name: group, dtype: int64
*****************************

********* LC-MS (-) *********
Dataset shape: (78, 533)
Number of features: 532
Number of missing values: 2660
Factor1:CASE       39
Factor1:CONTROL    39
Name: group, dtype: int64
*****************************



In [11]:
# Drop the missing values
df_np.dropna(inplace=True)
df_rn.dropna(inplace=True)

# Concatenate the dataframes based on index (samples); Keep common columns only once
df_np.drop('group', axis=1, inplace=True)   
df_rn.drop('group', axis=1, inplace=True)
df = pd.concat([df_np, df_rn, df_rp, gc], axis=1, join='inner')
# Reorder a column
df = df[[col for col in df.columns if col != 'group'] + ['group']]
df.head()


,1014.6391_19.2,102.0461_0.94,1020.7329_29.96,1026.743_24.96,1028.6159_19.2,1034.6116_1.02,1043.5805_19.2,1043.5815_18.42,1046.5904_19.2,1052.7966_35.63,...,Threitol,Threonine,trans-4-hydroxy-L-proline,Trans-aconitic acid,Tryptophan,Tyrosine,Urea,Uric acid,Valine,group
12,16876,239553,29039,18318,20935,3100,171854,17408,61443,212831,...,2197.53,24447.77,6405.79,1394.76,32045.44,65461.58,3860294.72,58341.17,179807.52,Factor1:CASE
15,24975,511854,10877,9677,27443,2279,219430,25666,78598,174309,...,1454.86,21656.99,6685.78,1795.11,35471.25,51331.44,3230542.69,49711.39,154433.49,Factor1:CASE
19,15809,295684,3738,3733,16316,2410,161321,14260,52448,234907,...,3799.36,25821.64,10985.81,2930.77,62623.37,95582.13,4912070.89,65151.55,258668.84,Factor1:CASE
2,14274,316320,4516,2945,18558,4198,149677,12523,48566,166061,...,3445.45,31701.67,14229.90,2779.67,30834.64,113637.54,3507068.54,89706.31,319385.40,Factor1:CASE
21,25926,240802,23787,18246,29704,4356,261080,38268,91711,220839,...,2466.10,18129.38,8825.26,2788.51,34267.17,43127.44,1521851.33,81950.87,178226.73,Factor1:CASE


In [12]:
print(f'Composite dataset shape: {df.shape}\n')
print(df['group'].value_counts())

Composite dataset shape: (71, 1431)

Factor1:CASE       36
Factor1:CONTROL    35
Name: group, dtype: int64


In [13]:
# Save the composite dataset
df.to_csv('data/composite_dataset.csv')

### NHS

In [14]:
# Read excel file with multiple sheets
xlsx = pd.ExcelFile('data/NHS_data.xlsx')
print(xlsx.sheet_names)

# Read the sheets into dataframes
df_nhs1 = pd.read_excel(xlsx, 'Figure 1 DN Control')
df_nhs2 = pd.read_excel(xlsx, 'Figure 1 Medicated Control')

['Figure 1 DN Control', 'Figure 1 Medicated Control', 'Figure 2 & SI Figure S2', 'Figure 3', 'Figure 4', 'SI Figure S1', 'SI Figure S3']


In [15]:
df_nhs1.set_index('repidx', inplace=True)   
df_nhs2.set_index('repidx', inplace=True)   

In [16]:
# LC-MS (+) - Healthy control
print('********* Healthy control *********')
print(f'Dataset shape: {df_nhs1.shape}')
print(f'Number of features: {df_nhs1.shape[1]-1}')
print(f'Number of missing values: {df_nhs1.isnull().sum().sum()}')
print(df_nhs1['label'].value_counts())
df_nhs1.to_csv('data/nhs_healthy.csv')
print('*****************************')
print()

# LC-MS (+) - Drug naïve 
print('********* Drug naïve *********')
print(f'Dataset shape: {df_nhs2.shape}')
print(f'Number of features: {df_nhs2.shape[1]-1}')
print(f'Number of missing values: {df_nhs2.isnull().sum().sum()}')
print(df_nhs2['label'].value_counts())
df_nhs2.to_csv('data/nhs_drug_naive.csv')
print('*****************************')
print()

********* Healthy control *********
Dataset shape: (136, 6503)
Number of features: 6502
Number of missing values: 0
2    80
1    56
Name: label, dtype: int64
*****************************

********* Drug naïve *********
Dataset shape: (194, 6503)
Number of features: 6502
Number of missing values: 0
3    138
1     56
Name: label, dtype: int64
*****************************

